# Dynamic Customer Support RAG chatbot 

This notebook demonstrates:

- Document ingestion
- Embedding generation
- FAISS vector indexing 
- Content retrieval 
- LLM-Based answer generation
- Incremental knowledge update